# Proyecto Final Analítica de Datos
##### Hecho por: 
##### * Juan Sebastián Clavijo Martínez (jclavijomartinez@gmail.com) - PUJ - Ing. en redes y telecomunicaciones
##### * Santiago Camacho (santiagocamachov@javeriana.edu.co ) - PUJ - Ing. de sistemas
##### * Juan Pablo González (gonzalez-juanp@javeriana.edu.co) - PUJ - Ing. de sistemas <br />
**Profesor: Jhon Corredor**<br />
Fecha: 14-11-2023<br />
Notas: <br />
Dataset: Datos de nacimiento de los pueblos de antioquia y chocó del 2009 a 2019 obtenidos de la pagína del DANE e información sobre areas deforestadas en el chocó obtenido de datos.gov.co

In [35]:
#Se importan las bibliotecas pertinentes para el desarrollo del proyecto
import pandas as pd
import numpy as np
import time ##Para medir rendimiento de los modelos
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
#Se importan las librerías pertinentes de PySpark 
import pyspark as pk
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col 
from pyspark.sql.types import IntegerType, FloatType

In [36]:
#Se crean dos dataframes, uno para cada set de datos. El primero corresponde al set de datos de areas deforestadas en el depaertamento del chocó en Colombia. El segundo corresponde a los datos de nacimiento de algunos municipios en Colombia. 
areas_deforestadas=pd.read_csv("https://raw.githubusercontent.com/jclavijomartinez/analiticaDeDatos2330/master/datos%20proyecto%20final/AREAS_DEFORESTADAS_CHOCO.csv")
# Para el dataframe de nacimientos se carga el archivo CSV especificando el delimitador como punto y coma ya que en el archivo CSV no se utiliza la coma como delimitador sino el ;
nacimientos = pd.read_csv("https://raw.githubusercontent.com/jclavijomartinez/analiticaDeDatos2330/master/datos%20proyecto%20final/finaldatosnacimientos.csv", sep=';')


In [37]:
#Se imprime la información del dataframe de areas deforestadas del chocó.
areas_deforestadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7937 entries, 0 to 7936
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              7937 non-null   int64  
 1   TIPO GEOMETRIA  7937 non-null   object 
 2   AÑO             7937 non-null   int64  
 3   IMAGEN          5901 non-null   object 
 4   CAUSA           7718 non-null   object 
 5   AREA_Ha         7937 non-null   float64
 6   OBSERVACION     7937 non-null   object 
 7   LATITUD         7930 non-null   object 
 8   LONGITUD        7930 non-null   object 
 9   MUNICIPIO       7937 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 620.2+ KB


In [38]:
#Se imprime la información del dataframde de nacimientos en los municipios del país.
nacimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       955 non-null    int64  
 1   departamento              955 non-null    object 
 2   municipio                 955 non-null    object 
 3   total                     955 non-null    float64
 4   total H                   955 non-null    object 
 5   total M                   955 non-null    float64
 6   total indet               955 non-null    object 
 7   cabecera municipal H      955 non-null    object 
 8   cabecera municipal M      955 non-null    float64
 9   cebecera municipal indet  955 non-null    object 
 10  centro poblado H          955 non-null    float64
 11  centro poblado M          955 non-null    float64
 12  centro poblado indet      955 non-null    object 
 13  rural disperso H          955 non-null    float64
 14  rural disp

##### OBSERVACIÓN: Se evidencia que en ambos dataframes los valores de las columnas no son los mismos, lo que indica que hay datos duplicados y/o faltantes. A continuación se realizará la limpieza de los datasets teniendo como objetivo eliminar los dataos faltantes y los datos duplicados. 

In [39]:
##Cuenta de datas null, etc... para el dataframe de nacimientos     
desaparecidos = len(nacimientos)-len(nacimientos.dropna())

print('Cantidad de observaciones con Datos NaN', desaparecidos)
print('Cantidad de datos duplicados', nacimientos.duplicated().sum())

Cantidad de observaciones con Datos NaN 192
Cantidad de datos duplicados 0


In [40]:
##Cuenta de datas null, etc... para el dataframe de areas deforestadas
desaparecidos = len(areas_deforestadas)-len(areas_deforestadas.dropna())

print('Cantidad de observaciones con Datos NaN', desaparecidos)
print('Cantidad de datos duplicados', areas_deforestadas.duplicated().sum())

Cantidad de observaciones con Datos NaN 2262
Cantidad de datos duplicados 0


In [41]:
## Se eliminan los datos Null y Duplicados para ambos dataframes
areas_deforestadas.dropna(inplace=True)
areas_deforestadas.drop_duplicates(inplace = True)
nacimientos.dropna(inplace=True)
nacimientos.drop_duplicates(inplace=True)
#Se reinicia el indice por los eliminados (duplicados)
areas_deforestadas.reset_index(drop = True, inplace = True)
nacimientos.reset_index(drop = True, inplace = True)

In [42]:
#Se imprime la nueva información del dataset de nacimientos por municipio
nacimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       763 non-null    int64  
 1   departamento              763 non-null    object 
 2   municipio                 763 non-null    object 
 3   total                     763 non-null    float64
 4   total H                   763 non-null    object 
 5   total M                   763 non-null    float64
 6   total indet               763 non-null    object 
 7   cabecera municipal H      763 non-null    object 
 8   cabecera municipal M      763 non-null    float64
 9   cebecera municipal indet  763 non-null    object 
 10  centro poblado H          763 non-null    float64
 11  centro poblado M          763 non-null    float64
 12  centro poblado indet      763 non-null    object 
 13  rural disperso H          763 non-null    float64
 14  rural disp

In [43]:
#Se imprime la nueva información del dataset de areas deforestadas
areas_deforestadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5675 entries, 0 to 5674
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              5675 non-null   int64  
 1   TIPO GEOMETRIA  5675 non-null   object 
 2   AÑO             5675 non-null   int64  
 3   IMAGEN          5675 non-null   object 
 4   CAUSA           5675 non-null   object 
 5   AREA_Ha         5675 non-null   float64
 6   OBSERVACION     5675 non-null   object 
 7   LATITUD         5675 non-null   object 
 8   LONGITUD        5675 non-null   object 
 9   MUNICIPIO       5675 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 443.5+ KB


##### OBSERVACIÓN: Se evidencia que se eliminaron los registros duplicados y valores faltantes para ambos datasets. Es decir, todas las columnas de ambos datasets tienen exactamente el mismo número de registros. 

##### nota: se cambiarán los nombres de algunas columnas de los datasets para que cumplan con el estándar PEP8

In [44]:
#Se cambian los nombres de ciertas columnas para que cumplan el estándar PEP8
areas_deforestadas = areas_deforestadas.rename(columns={'TIPO GEOMETRIA': 'TIPO_GEOMETRIA'})
nacimientos = nacimientos.rename(columns={'total H': 'total_H'})
nacimientos = nacimientos.rename(columns={'total M': 'total_M'})
nacimientos = nacimientos.rename(columns={'total indet': 'total_indet'})
nacimientos = nacimientos.rename(columns={'cabecera municipal H': 'cabecera_municipal_H'})
nacimientos = nacimientos.rename(columns={'cabecera municipal M': 'cabecera_municipal_M'})
nacimientos = nacimientos.rename(columns={'cebecera municipal indet': 'cabecera_municipal_indet'})
nacimientos = nacimientos.rename(columns={'centro poblado H': 'centro_poblado_H'})
nacimientos = nacimientos.rename(columns={'centro poblado M': 'centro_poblado_M'})
nacimientos = nacimientos.rename(columns={'centro poblado indet': 'centro_poblado_indet'})
nacimientos = nacimientos.rename(columns={'rural disperso H': 'rural_disperso_H'})
nacimientos = nacimientos.rename(columns={'rural disperso M': 'rural_disperso_M'})
nacimientos = nacimientos.rename(columns={'rural disperso indet': 'rural_disperso_indet'})
nacimientos = nacimientos.rename(columns={'sin info H': 'sin_info_H'})
nacimientos = nacimientos.rename(columns={'sin info M': 'sin_info_M'})
nacimientos = nacimientos.rename(columns={'sin info indet': 'sin_info_indet'})




In [45]:
#Se imprime la nueva información del dataset de areas deforestadas
areas_deforestadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5675 entries, 0 to 5674
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              5675 non-null   int64  
 1   TIPO_GEOMETRIA  5675 non-null   object 
 2   AÑO             5675 non-null   int64  
 3   IMAGEN          5675 non-null   object 
 4   CAUSA           5675 non-null   object 
 5   AREA_Ha         5675 non-null   float64
 6   OBSERVACION     5675 non-null   object 
 7   LATITUD         5675 non-null   object 
 8   LONGITUD        5675 non-null   object 
 9   MUNICIPIO       5675 non-null   object 
dtypes: float64(1), int64(2), object(7)
memory usage: 443.5+ KB


In [46]:
#Se imprime la nueva información del dataset de nacimientos
nacimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   año                       763 non-null    int64  
 1   departamento              763 non-null    object 
 2   municipio                 763 non-null    object 
 3   total                     763 non-null    float64
 4   total_H                   763 non-null    object 
 5   total_M                   763 non-null    float64
 6   total_indet               763 non-null    object 
 7   cabecera_municipal_H      763 non-null    object 
 8   cabecera_municipal_M      763 non-null    float64
 9   cabecera_municipal_indet  763 non-null    object 
 10  centro_poblado_H          763 non-null    float64
 11  centro_poblado_M          763 non-null    float64
 12  centro_poblado_indet      763 non-null    object 
 13  rural_disperso_H          763 non-null    float64
 14  rural_disp

Se imprimen los primeros 15 registros de cada dataframe

In [47]:
nacimientos.head(5)

,año,departamento,municipio,total,total_H,total_M,total_indet,cabecera_municipal_H,cabecera_municipal_M,cabecera_municipal_indet,centro_poblado_H,centro_poblado_M,centro_poblado_indet,rural_disperso_H,rural_disperso_M,rural_disperso_indet,sin_info_H,sin_info_M,sin_info_indet
0,2009,antioquia,total,85.955,44.173,41.782,"0,00",33.222,31.279,"0,00",3.249,3.032,"0,00",6.393,6.137,"0,00",1.309,1.334,"0,00"
1,2009,antioquia,MEDELLÍN,31.893,16.334,15.559,"0,00",15.871,15.141,"0,00",210.000,208.000,"0,00",126.000,101.000,"0,00",127.000,109.000,"0,00"
2,2009,antioquia,ABEJORRAL,271.000,137,134.000,"0,00",50,43.000,"0,00",9.000,5.000,"0,00",67.000,74.000,"0,00",11.000,12.000,"0,00"
3,2009,antioquia,ABRIAQUÍ,21.000,11,10.000,"0,00",4,6.000,"0,00",1.000,1.000,"0,00",6.000,3.000,"0,00",0.000,0.000,"0,00"
4,2009,antioquia,ALEJANDRÍA,44.000,17,27.000,"0,00",4,14.000,"0,00",0.000,0.000,"0,00",13.000,11.000,"0,00",0.000,2.000,"0,00"


In [48]:
areas_deforestadas.head(5)

,ID,TIPO_GEOMETRIA,AÑO,IMAGEN,CAUSA,AREA_Ha,OBSERVACION,LATITUD,LONGITUD,MUNICIPIO
0,7463,Polygon,2020,T18PTQ_20200201T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.126512,Ninguna,"8° 22' 43,726"" N","77° 14' 19,019"" W",ACANDÍ
1,4659,Polygon,2019,T18PTQ_20190527T153621_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.140720,Ninguna,"8° 30' 47,551"" N","77° 19' 30,507"" W",ACANDÍ
2,7459,Polygon,2020,T18PTQ_20200112T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.141250,Ninguna,"8° 22' 30,966"" N","77° 13' 49,720"" W",ACANDÍ
3,9719,Polygon,2021,T18PTQ_20201202T153619_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.158066,Ninguna,"8° 36' 11.256"" N","77° 20' 50.757"" W",ACANDÍ
4,7464,Polygon,2020,T18PTQ_20200201T153611_TCI_10m.jp2 - T18PTQ_20...,Cultivo,0.184549,Ninguna,"8° 22' 22,008"" N","77° 14' 3,921"" W",ACANDÍ


In [49]:
# ya que los dfs están listos para ser usados, 
# se crea uno nuevo con la información de los pueblos a ser evaluados, se va a usar primero arboles de desicion como modelo de ML, 
# se vizualiza la informacion del nuevo df con los datos refinados
municipios_interes=['ZARAGOZA','NECHÍ','NÓVITA']
df_seleccion = nacimientos[nacimientos['municipio'].isin(municipios_interes)]
df_seleccion.head(5)

,año,departamento,municipio,total,total_H,total_M,total_indet,cabecera_municipal_H,cabecera_municipal_M,cabecera_municipal_indet,centro_poblado_H,centro_poblado_M,centro_poblado_indet,rural_disperso_H,rural_disperso_M,rural_disperso_indet,sin_info_H,sin_info_M,sin_info_indet
75,2009,antioquia,NECHÍ,595.0,322,273.0,"0,00",179,142.0,"0,00",26.0,27.0,"0,00",94.0,74.0,"0,00",23.0,30.0,"0,00"
125,2009,antioquia,ZARAGOZA,581.0,315,266.0,"0,00",180,151.0,"0,00",19.0,18.0,"0,00",93.0,74.0,"0,00",23.0,23.0,"0,00"
656,2009,choco,NÓVITA,48.0,21,27.0,0,16,16.0,0,1.0,5.0,0,4.0,2.0,0,0.0,4.0,0
720,2011,choco,NÓVITA,48.0,21,27.0,0,16,16.0,0,1.0,5.0,0,4.0,2.0,0,0.0,4.0,0
752,2013,choco,NÓVITA,48.0,21,27.0,0,16,16.0,0,1.0,5.0,0,4.0,2.0,0,0.0,4.0,0


In [50]:
# se asignan las caracrerísticas y el objetivo (variable a predecir)
caracteristicas = ['departamento','total_H','total_M','total_indet']
objetivo = 'total'
X  = df_seleccion[caracteristicas]
y = df_seleccion[objetivo]

In [51]:
#se divide el conjunto en datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
# Creamos el modelo
modelo_arbol = DecisionTreeClassifier(random_state=42)

# Entrenamos el modelo
modelo_arbol.fit(X_train, y_train)

ValueError: could not convert string to float: 'choco'

In [ ]:
# Hacemos predicciones en el conjunto de prueba
predicciones = modelo_arbol.predict(X_test)

In [ ]:
# Evaluamos la precisión
precision = accuracy_score(y_test, predicciones)
print(f"Precisión del modelo: {precision}")

# Mostramos la matriz de confusión
matriz_confusion = confusion_matrix(y_test, predicciones)
print("Matriz de Confusión:")
print(matriz_confusion)

# Mostramos el reporte de clasificación
reporte_clasificacion = classification_report(y_test, predicciones)
print("Reporte de Clasificación:")
print(reporte_clasificacion)